# Kaggle House Price Prediction

In [1]:
%matplotlib inline
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import sys
import renyan_utils as ry

In [2]:
print(torch.__version__)

1.6.0


In [3]:
torch.set_default_tensor_type(torch.FloatTensor)

## Data

Get & Preprocess

In [4]:
train_data = pd.read_csv('data/KaggleHousePrice/train.csv')
test_data = pd.read_csv('data/KaggleHousePrice/test.csv')

In [5]:
train_data.shape

(1460, 81)

In [6]:
test_data.shape

(1459, 80)

In [72]:
all_features = pd.concat((train_data.iloc[:,1:-1], test_data.iloc[:,1:]))

In [73]:
all_features.shape

(2919, 79)

In [74]:
# all dtypes: O(object), int64, float64
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index

In [75]:
all_features.dtypes

MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
                  ...   
MiscVal            int64
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
Length: 79, dtype: object

In [76]:
set(all_features.dtypes.values)

{dtype('int64'), dtype('float64'), dtype('O')}

In [77]:
all_features[numeric_features] = all_features[numeric_features].apply(lambda x: (x-x.mean())/x.std())
all_features = all_features.fillna(0)

In [78]:
# all_features.isnull().any()
# print(all_features[all_features.isnull().values==True].shape, 
# all_features.shape)
# (all_features[all_features.isnull().values==True]).shape

In [83]:
# dummy(Na is a class as well)
# every class has a corresponding column(not like the mute varibales)
all_features = pd.get_dummies(all_features, dummy_na = True)
all_features.shape

(2919, 354)

In [97]:
n_train = train_data.shape[0]
train_features = torch.tensor(all_features[:n_train].values, dtype = torch.float)
test_features = torch.tensor(all_features[n_train:].values, dtype = torch.float)
train_labels = torch.tensor(train_data.SalePrice.values)

## Model

In [89]:
def log_rmse(net, features, labels):
    with torch.no_grad():
        clipped_preds = torch.max(net(features), torch.tensor(1.0))
        rmse = torch.sqrt(2 * loss(clipped_preds.log(), lables.log()).mean())
    return rmse.item()

In [90]:
def train(net, train_features, train_labels, test_features, test_labels, nu_epochs, learning_rate, weight_decay, batch_size):
    train_ls, test_ls = [], []
    dataset = torch.utils.data.TensorDataset(train_features, train_labels)
    train_iter = torch.utils.data.DataLoader(dataset, batch_size, shuffle = True)
    
    optimizer = torch.optim.Adam(params = net.parameters(), lr = learning_rate, weight_decay = weight_decay)
    net = net.float()
    for epoch in range(num_epochs):
        for X, y in train_iter:
            l = loss(net(X.float()), y.float())
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
        train_ls.append(log_rmse(net, train_features, train_labels))
        if test_labels is not None:
            test_ls.append(log_tmse(net, test_features, test_labels))
    
    return train_ls, test_ls

In [91]:
def get_k_fold_data(k, i, X, y):
    assert k > 1
    fold_size = X.shape[0] // k
    X_train, y_train = None, None
    for j in range(k):
        idx = slice(j * fold_size, (j + 1) * fold_size)
        X_part, y_part = X[idx, :], y[idx]
        if j == i:
            X_valid, y_valid = X_part, y_part
        elif X_train is None:
            X_train, y_train = X_part, y_part
        else:
            X_train = torch.cat((X_train, X_part), dim=0)
            y_train = torch.cat((y_train, y_part), dim=0)
            
    return X_train, y_train, X_valid, y_valid

In [ ]:
def semilogy(x_vals, y_vals, x_label, y_label, x2_vals=None,y2_vals=None,legend=None, figsize=(3.5, 2.5)):
d2l.set_figsize(figsize)
d2l.plt.xlabel(x_label)
d2l.plt.ylabel(y_label)
d2l.plt.semilogy(x_vals, y_vals)
if x2_vals and y2_vals:
d2l.plt.semilogy(x2_vals, y2_vals, linestyle=':')
d2l.plt.legend(legend)

In [92]:
def k_fold(k, X_train, y_train, num_epochs, learning_rate, weight_decay, batch_size):
    train_l_sum, valid_l_sum = 0, 0
    
    for i in range(k):
        data = get_k_fold_data(k, i, X_train, y_train)
        net = get_net(X_train.shape[1])
        train_ls, valid_ls = train(net, *data, num_epochs, learning_rate, weight_decay, batch_size)
        train_l_sum += train_ls[-1]
        valid_l_sum += valid_ls[-1]
#         if i == 0:
#             semilogy(range(1, num_epochs + 1), train_ls,'epochs', 'rmse', range(1, num_epochs + 1), valid_ls,['train', 'valid'])
        print('fold %d, train rmse %f, valid rmse %f' % (i,
                                                             train_ls[-1], valid_ls[-1]))
    return train_l_sum / k, valid_l_sum / k

In [99]:
k, num_epochs, lr, weight_decay, batch_size = 5, 100, 5, 0, 64
train_l, valid_l = k_fold(k, train_features, train_labels, num_epochs, lr, weight_decay, batch_size)
print('%d-fold validation: avg train rmse %f, avg valid rmse %f' % (k, train_l, valid_l))

NameError: name 'get_net' is not defined

In [98]:
train_labels

tensor([208500, 181500, 223500,  ..., 266500, 142125, 147500])